In [6]:
import pandas as pd
from os.path import expanduser
import matplotlib.pyplot as plt


In [7]:
experiment_name = 'P3856'
run_name = 'P3856_YHE211_1_Slot1-1_1_5104'
precursor_definition_method = 'pasef'

In [8]:
MAXIMUM_Q_VALUE = 0.01

In [9]:
BASE_RESULTS_DIR = '/media/big-ssd/results-{}'.format(experiment_name)

In [12]:
BASE_DIR = '{}/P3856-results-cs-false-fmdw-false-2021-05-18-18-07-57'.format(BASE_RESULTS_DIR)

FEATURES_DIR = '{}/features-{}'.format(BASE_DIR, precursor_definition_method)
FEATURES_NAME = '{}/exp-{}-run-{}-features-{}.pkl'.format(FEATURES_DIR, experiment_name, run_name, precursor_definition_method)
FEATURES_DEDUP_NAME = '{}/exp-{}-run-{}-features-{}-dedup.pkl'.format(FEATURES_DIR, experiment_name, run_name, precursor_definition_method)

IDENTS_DIR = '{}/identifications-{}'.format(BASE_DIR, precursor_definition_method)
IDENTS_NAME = '{}/exp-{}-identifications-{}-recalibrated.pkl'.format(IDENTS_DIR, experiment_name, precursor_definition_method)

In [13]:
features_before_dedup_df = pd.read_pickle(FEATURES_NAME)['features_df']

In [14]:
features_after_dedup_df = pd.read_pickle(FEATURES_DEDUP_NAME)['features_df']

In [15]:
identifications_before_unique_df = pd.read_pickle(IDENTS_NAME)['identifications_df']

In [16]:
unique_sequences_l = []
for group_name,group_df in identifications_before_unique_df.groupby(['sequence','charge'], as_index=False):
    min_q_value = group_df['percolator q-value'].min()
    min_mass_accuracy_ppm = group_df['mass_accuracy_ppm'].min()
    unique_sequences_l.append({'sequence_charge':group_name, 'min_mass_accuracy_ppm':min_mass_accuracy_ppm, 'min_q_value':min_q_value, 'feature_ids':group_df.feature_id.tolist(), 'q-values':group_df['percolator q-value'].tolist()})
unique_sequences_df = pd.DataFrame(unique_sequences_l)

In [17]:
unique_sequences_below_min_q_df = unique_sequences_df[(unique_sequences_df.min_q_value <= MAXIMUM_Q_VALUE)]

In [18]:
unique_sequences_below_min_ppm_df = unique_sequences_df[(abs(unique_sequences_df.min_mass_accuracy_ppm) <= 20)]

In [19]:
stats_d = {'features_before_dedup':len(features_before_dedup_df), 'features_after_dedup':len(features_after_dedup_df), 'recalibrated_idents_total':len(identifications_before_unique_df), 'recalibrated_idents_unique':len(unique_sequences_df), 'unique_idents_below_min_qvalue':len(unique_sequences_below_min_q_df), 'unique_sequences_below_min_ppm':len(unique_sequences_below_min_ppm_df)}
stats_d

{'features_before_dedup': 101985,
 'features_after_dedup': 73662,
 'recalibrated_idents_total': 37510,
 'recalibrated_idents_unique': 35881,
 'unique_idents_below_min_qvalue': 7913,
 'unique_sequences_below_min_ppm': 35880}

In [20]:
features_after_dedup_df.columns

Index(['mono_mz_lower', 'mono_mz_upper', 'scan_apex', 'scan_lower',
       'scan_upper', 'rt_apex', 'rt_lower', 'rt_upper',
       'intensity_without_saturation_correction',
       'intensity_with_saturation_correction',
       'mono_intensity_adjustment_outcome',
       'mono_mz_without_saturation_correction',
       'mono_mz_with_saturation_correction', 'isotopic_peaks', 'scan_df',
       'rt_df', 'monoisotopic_mz', 'charge', 'monoisotopic_mass',
       'feature_intensity', 'envelope', 'isotope_count', 'deconvolution_score',
       'precursor_cuboid_id', 'fragment_ions_l', 'fmdw_before_after_d',
       'feature_id', 'run_name'],
      dtype='object')

In [22]:
type(features_after_dedup_df.iloc[0].isotopic_peaks)

list

In [23]:
s1 = features_after_dedup_df.isotope_count.value_counts(normalize=False).rename('counts', inplace=True)
s2 = round(features_after_dedup_df.isotope_count.value_counts(normalize=True)*100,1).rename('%', inplace=True)
pd.concat([s1, s2], axis=1)

,counts,%
4,37683,51.2
5,18430,25.0
3,12945,17.6
6,3536,4.8
7,733,1.0
8,196,0.3
9,113,0.2
10,19,0.0
11,6,0.0
13,1,0.0
